In [458]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [459]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [460]:
len(words)

32033

In [461]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [462]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []

for w in words:
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    context = context[1:] + [ix] # rolling window

X = torch.tensor(X)
Y = torch.tensor(Y)

In [463]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [464]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [465]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])

In [466]:
# embeddings table
# take vectors and embed them in lower dimensional space
# we have 27 possible characters, we put them in 2-dimensional space

C = torch.randn((27, 2)) # embeddings table

In [467]:
C[5] # this embeds an integer into 2 dimensional vector 

tensor([0.6501, 0.1645])

In [468]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C # this embeds an integer into 2 dimensional vector 

tensor([0.6501, 0.1645])

In [469]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [470]:
# neural network first layer (hidden layer)
W1 = torch.randn((6, 100)) # weights
b1 = torch.randn(100) # biases

In [471]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer with activation function
h

tensor([[ 0.8132, -0.9352,  0.3975,  ...,  0.8103, -0.9771,  0.2867],
        [ 0.9744, -0.2575,  0.9491,  ...,  0.9443, -0.9924, -0.5562],
        [ 0.8876, -0.9889, -0.6509,  ...,  0.5778, -0.9842,  0.9673],
        ...,
        [-0.9440, -0.8353,  0.0906,  ..., -0.5778, -0.8930, -0.8463],
        [-0.9461, -0.9392, -0.4878,  ..., -0.6782, -0.8806, -0.5025],
        [-0.9928, -0.2960, -0.0196,  ..., -0.8759, -0.5153, -0.3993]])

In [472]:
# neural network second layer (output layer)
W2 = torch.randn((100, 27)) # weights
b2 = torch.randn(27) # biases

In [473]:
# NOW REFACTORED MLP

In [474]:
# dataset
# X - trigrams of characters
# Y - next character that comes after the trigram
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [475]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g) # embeddings table: it basically converts any given input character into a 2 dimensional vector
# neural network first layer (hidden layer)
W1 = torch.randn((6, 100), generator=g) # weights
b1 = torch.randn(100, generator=g) # biases
# neural network second layer (output layer)
W2 = torch.randn((100, 27), generator=g) # weights
b2 = torch.randn(27, generator=g) # biases
parameters = [C, W1, b1, W2, b2]

In [476]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [477]:
for p in parameters:
  p.requires_grad = True

In [478]:
# lre = torch.linspace(-3, 0, 1000)
# lrs = 10**lre
# lrs

In [491]:
# training
lri = []
lossi = []

for i in range(10000):
  # minibatch construct
  ix = torch.randint(0, X.shape[0], (32, ))

  # forward pass
  emb = C[X[ix]] # (32, 3, 2), we select only 32 examples from a random minibatch
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer with activation function, (32, 100)
  logits = h @ W2 +   b2
  loss = F.cross_entropy(logits, Y[ix])

  # backward pass
  for p in parameters:
    p.grad = None 
  loss.backward()

  # update
  lr = 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # # track stats
  # lri.append(lr)
  # lossi.append(loss.item())

print(loss.item())


2.1608164310455322


In [492]:
# loss for the whole dataset
emb = C[X] # (32, 3, 2), we select only 32 examples from a random minibatch
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer with activation function, (32, 100)
logits = h @ W2 +   b2
loss = F.cross_entropy(logits, Y)
loss

tensor(2.3286, grad_fn=<NllLossBackward0>)